In [32]:
from unsloth import FastLanguageModel
import torch
from vllm import SamplingParams
from trl import GRPOConfig, GRPOTrainer

In [33]:
max_seq_length = 1024

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-3.2-1B-Instruct", # Change to any model in fourbit_models!
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    fast_inference= True,  # Enable fast inference
    # token = "hf_...", # use one if using gated models
)

Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.9.1: Fast Llama patching. Transformers: 4.55.4. vLLM: 0.10.1.1.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Llama-3.2-1B-Instruct with actual GPU utilization = 17.66%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 23.99 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 1.85 GB. Also swap space = 2 GB.
Unsloth: Not an error, but `device` is not supported in vLLM. Skipping.
INFO 09-05 13:30:19 [utils.py:326] non-defau

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.92it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.92it/s]


INFO 09-05 13:30:22 [default_loader.py:262] Loading weights took 0.55 seconds


INFO 09-05 13:30:23 [gpu_model_runner.py:2007] Model loading took 2.4200 GiB and 1.017143 seconds
INFO 09-05 13:30:26 [backends.py:548] Using cache directory: /home/miguel/.cache/vllm/torch_compile_cache/b49c5968bd/rank_0_0/backbone for vLLM's torch.compile
INFO 09-05 13:30:26 [backends.py:559] Dynamo bytecode transform time: 3.05 s
INFO 09-05 13:30:28 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.925 s
INFO 09-05 13:30:29 [monitor.py:34] torch.compile takes 3.05 s in total
INFO 09-05 13:30:29 [gpu_worker.py:276] Available KV cache memory: 1.23 GiB
INFO 09-05 13:30:29 [kv_cache_utils.py:849] GPU KV cache size: 40,304 tokens
INFO 09-05 13:30:29 [kv_cache_utils.py:853] Maximum concurrency for 1,024 tokens per request: 39.36x
INFO 09-05 13:30:30 [vllm_utils.py:667] Unsloth: Running patched vLLM v1 `capture_model`.
INFO 09-05 13:30:30 [vllm_utils.py:667] Unsloth: Running patched vLLM v1 `capture_model`.


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 35/35 [00:02<00:00, 13.63it/s]

INFO 09-05 13:30:32 [gpu_model_runner.py:2708] Graph capturing finished in 3 secs, took 1.65 GiB
INFO 09-05 13:30:32 [vllm_utils.py:674] Unsloth: Patched vLLM v1 graph capture finished in 3 secs.
INFO 09-05 13:30:32 [vllm_utils.py:674] Unsloth: Patched vLLM v1 graph capture finished in 3 secs.


INFO 09-05 13:30:33 [core.py:214] init engine (profile, create kv cache, warmup model) took 10.13 seconds
INFO 09-05 13:30:33 [llm.py:298] Supported_tasks: ('generate',)
Unsloth: Just some info: will skip parsing ['k_norm', 'pre_feedforward_layernorm', 'post_feedforward_layernorm', 'q_norm']
Unsloth: Just some info: will skip parsing ['k_norm', 'pre_feedforward_layernorm', 'post_feedforward_layernorm', 'q_norm']


In [34]:
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
print(system_prompt)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION>


In [35]:
problem = "A notebook costs **3 €** and a book costs **12 €**. A student buys **3 notebooks and 2 books**, paying with a **50 € bill**. How much money does the student spend in total? How much change does the student get back? If instead of 2 books the student buys **x books**, write an expression that represents the total cost."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": problem},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 05 Sep 2025

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|eot_id|><|start_header_id|>user<|end_header_id|>

A notebook costs **3 €** and a book costs **12 €**. A student buys **3 notebooks and 2 books**, paying with a **50 € bill**. How much money does the student spend in total? How much change does the student get back? If instead of 2 books the student buys **x books**, write an expression that represents the total cost.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<start_working_out>
A notebook costs **3 €** and a book costs **12 €**. A student buys **3 notebooks and 2 books**, paying with a **50 € bill**.
To find the total cost, we need to calculate the cost of the notebooks and the books separ

In [36]:
model = FastLanguageModel.get_peft_model(
    model,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

In [37]:
from datasets import load_dataset
dataset = load_dataset("openai/gsm8k", "main", split = "train")
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

In [38]:
print("Question:", dataset[0]["question"])
print("Answer:", dataset[0]["answer"])

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer: Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72


In [39]:
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
print(extract_hash_answer(dataset[0]["answer"]))

72


In [40]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": extract_hash_answer(x["answer"]),
})
tiny_dataset = dataset.select(range(16)) # For testing, use full dataset for real training!
dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72',
 'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>',
   'role': 'system'},
  {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
   'role': 'user'}]}

In [41]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [42]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

<re.Match object; span=(0, 71), match='<start_working_out>Let me think!<end_working_out>>

In [43]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

In [44]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

In [45]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # Correct answer gets 3 points!
        if guess == true_answer:
            score += 3.0
        # Match if spaces are seen
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

In [46]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>")

['0.34']

In [47]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            guess       = float(guess.strip())
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0)
            continue
    return scores

In [48]:
maximum_length = 256
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [49]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = tiny_dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16 | Num Epochs = 25 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 5,636,096 of 1,241,450,496 (0.45% trained)


******************** Question:
Tobias is buying a new pair of shoes that costs $95. He has been saving up his money each month for the past three months. He gets a $5 allowance a month. He also mows lawns and shovels driveways. He charges $15 to mow a lawn and $7 to shovel. After buying the shoes, he has $15 in change. If he mows 4 lawns, how many driveways did he shovel? 
Answer:
5 
Response:
<start_working_out>

Tobias's earnings from mowing lawns:

4 lawns * $15 per lawn = $60

Earnings from mowing lawns + allowance = $60 + $5 per month = $65

Since Tobias started saving his money 3 months ago and spent $95 on the shoes.

= $65 - $95 = -$30

So, Tobias earned amounted $35 from mowing lawns.

Let's divide his earnings from mowing by the lawn

$35 ÷ $15 per lawn = 2.33 

Since you can't shovel a fraction of a driveway, Tobias must have shoveled 3 lawns to have enough money for the shoes.

SOLUTION> 
Extracted:
None


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,-0.000000,-1.125000,0.348462,135.625000,59.000000,241.000000,0.000000,135.625000,59.000000,241.000000,0.000000,0.000000,0.000000,-1.125000,0.500000,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.937500,0.329124,124.312500,45.000000,357.000000,0.000000,124.312500,45.000000,357.000000,0.000000,0.000000,0.000000,-0.937500,0.442531,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.750000,0.500000,128.687500,60.000000,267.000000,0.000000,128.687500,60.000000,267.000000,0.001308,0.000000,0.000000,-0.750000,0.577350,0.000000,0.000000,0.000000,0.000000
4,0.000000,-1.062500,0.375000,124.000000,35.000000,284.000000,0.000000,124.000000,35.000000,284.000000,0.001286,0.000000,0.000000,-1.062500,0.442531,0.000000,0.000000,0.000000,0.000000
5,0.000000,-1.250000,0.454124,138.125000,45.000000,267.000000,0.000000,138.125000,45.000000,267.000000,0.002395,0.000000,0.000000,-1.250000,0.683130,0.000000,0.000000,0.000000,0.000000
6,0.000000,-0.937500,0.579124,135.812500,52.000000,329.000000,0.000000,135.812500,52.000000,329.000000,0.002287,0.000000,0.000000,-0.937500,0.680074,0.000000,0.000000,0.000000,0.000000
7,0.000000,-1.062500,0.269338,97.062500,41.000000,248.000000,0.000000,97.062500,41.000000,248.000000,0.001994,0.000000,0.000000,-1.062500,0.442531,0.000000,0.000000,0.000000,0.000000
8,0.000000,-1.250000,0.712819,97.375000,33.000000,175.000000,0.000000,97.375000,33.000000,175.000000,0.001916,0.000000,0.000000,-1.250000,0.683130,0.000000,0.000000,0.000000,0.000000
9,0.000000,-0.937500,0.677586,94.875000,40.000000,149.000000,0.000000,94.875000,40.000000,149.000000,0.001834,0.000000,0.000000,-0.937500,0.680074,0.000000,0.000000,0.000000,0.000000
10,0.000000,-1.062500,0.473462,157.000000,53.000000,620.000000,0.000000,157.000000,53.000000,620.000000,0.002402,0.000000,0.000000,-1.062500,0.771902,0.000000,0.000000,0.000000,0.000000


******************** Question:
Ken created a care package to send to his brother, who was away at boarding school.  Ken placed a box on a scale, and then he poured into the box enough jelly beans to bring the weight to 2 pounds.  Then, he added enough brownies to cause the weight to triple.  Next, he added another 2 pounds of jelly beans.  And finally, he added enough gummy worms to double the weight once again.  What was the final weight of the box of goodies, in pounds? 
Answer:
16 
Response:
<start_working_out>
A box of 2 pounds
2 + (2 * 3) = 2 + 6 = 8
8 + 2 = 10
10 + (2 * 2) = 10 + 4 = 14
14 + 2 = 16

SOLUTION> 
Extracted:
None
******************** Question:
Alexis is applying for a new job and bought a new set of business clothes to wear to the interview. She went to a department store with a budget of $200 and spent $30 on a button-up shirt, $46 on suit pants, $38 on a suit coat, $11 on socks, and $18 on a belt. She also purchased a pair of shoes, but lost the receipt for them. S

TrainOutput(global_step=100, training_loss=9.984206230910787e-06, metrics={'train_runtime': 300.3758, 'train_samples_per_second': 5.327, 'train_steps_per_second': 0.333, 'total_flos': 0.0, 'train_loss': 9.984206230910787e-06})

In [50]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "In a store, each notebook costs 3 € and each pen costs 2 €. A student buys 4 notebooks and 5 pens.How much does the student pay in total?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.0,
    do_sample = False,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<start_working_out>
To find the total cost, we need to calculate the cost of notebooks and pens separately and then add them together.

Cost of notebooks = Number of notebooks × Cost per notebook
= 4 × 3
= 12 €

Cost of pens = Number of pens × Cost per pen
= 5 × 2
= 10 €

Total cost = Cost of notebooks + Cost of pens
= 12 + 10
= 22 €

SOLUTION><|eot_id|>


In [51]:
text = "In a store, each notebook costs 3 € and each pen costs 2 €. A student buys 4 notebooks and 5 pens.How much does the student pay in total?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.0,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

print(output)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.02s/it, est. speed input: 9.44 toks/s, output: 245.07 toks/s]

 The student pays 3.5 € for the notebooks and 5.5 € for the pens. The total amount paid by the student is 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student pays 3.5 + 5.5 = 9.0 €. The student

In [52]:
model.save_lora("grpo_saved_lora")

In [53]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

In [54]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "In a store, each notebook costs 3 € and each pen costs 2 €. A student buys 4 notebooks and 5 pens.How much does the student pay in total?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.0,
    seed=3407,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

print(output)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.51it/s, est. speed input: 175.71 toks/s, output: 193.88 toks/s]

<start_working_out>
The student buys 4 notebooks and 5 pens. Each notebook costs 3 € and each pen costs 2 €. To find the total cost, we need to calculate the cost of notebooks and pens separately and then add them together.

Cost of notebooks = Number of notebooks × Cost per notebook
= 4 × 3
= 12 €

Cost of pens = Number of pens × Cost per pen
= 5 × 2
= 10 €

Total cost = Cost of notebooks + Cost of pens
= 12 + 10
= 22 €

SOLUTION>
